In [19]:
import folium
import os
import sys

currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from hashed_similarities.grid_similarity import (
    generate_grid_hash_similarity_coordinates,
)

import folium

# Rome coordinates
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88
ROME_META_FILE = "../dataset/rome/output/META-50.txt"

# Initialize a Folium map at a central point
map = folium.Map(location=[(R_MAX_LAT + R_MIN_LAT) / 2, (R_MAX_LON + R_MIN_LON) / 2], zoom_start=13, tiles=None)

# Define the corners of the rectangle
corners = [
    (R_MAX_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MAX_LON)  # Repeat the first corner to close the rectangle
]

folium.Rectangle(
    bounds=[(R_MIN_LAT, R_MIN_LON), (R_MAX_LAT, R_MAX_LON)],
    color='black',
).add_to(map)



def read_coordinates(file_path):
    coordinates = []
    with open("../dataset/rome/output/" + file_path, "r") as file:
        count = 0
        for line in file:
            count += 1
            lat, lon = line.strip().split(", ")
            coordinates.append((float(lat), float(lon)))
        # print(f"Number of coordinates for {file}: {str(count)}")
    return coordinates


# Read the index file to get the list of files containing coordinates
with open(ROME_META_FILE, "r") as index_file:
    number_of_trajectories = 0
    for file_name in index_file:
        if number_of_trajectories == 2:
            break
        file_name = file_name.strip()  # Remove newline characters

        coordinates = read_coordinates(file_name)
        # Plot each coordinate on the map
        for lat, lon in coordinates:
            folium.CircleMarker(
                [lat, lon], radius=1, color="black", fill=False, fill_color="red"
            ).add_to(map)
        folium.PolyLine(coordinates, color="black", weight=1, opacity=1).add_to(map)
        number_of_trajectories += 1

hashed_coordinates, grid = generate_grid_hash_similarity_coordinates(
    city="rome", res=0.6, layers=2, measure="dtw", size=50
)

# TODO: use constant from constants when merged
colors = [
    "red",
    "blue",
    "green",
    "purple",
    "orange",
    "darkred",
    "lightblue",
    "beige",
    "darkgreen",
    "cadetblue",
]

for layer in grid:
    color = colors[layer]
    latitudes = grid[layer][0]
    longitudes = grid[layer][1]
    # print(f"Layer {layer} latitudes: {latitudes}")
    # print(f"Layer {layer} longitudes: {longitudes}")

    for lat in latitudes:
        for lon in longitudes:
            folium.CircleMarker(
                [lat, lon],
                radius=2,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=1,
            ).add_to(map)
    for lat in latitudes:
        polyline = [(lat, lon) for lon in longitudes]
        folium.PolyLine(polyline, color=color, weight=1).add_to(map)

    # Create polylines for longitude lines (vertical grid lines)
    for lon in longitudes:
        polyline = [(lat, lon) for lat in latitudes]
        folium.PolyLine(polyline, color=color, weight=1).add_to(map)

# print(f"\nNumber of hashed coordinates: {len(hashed_coordinates)}")
# print("Hashed coordinates:", hashed_coordinates)
number_of_hashed_trajectories = 0
for trajectory_key, trajectory_hash in hashed_coordinates.items():
    if number_of_hashed_trajectories == 2:
        break
    print(f"Trajectory hash: {trajectory_key} -> {trajectory_hash}")
    # print("len hash", len(trajectory_hash))
    layer = 0
    for hashed_trajectory_layer in trajectory_hash:
        print(
            f"Length of hashed coordinate in layer {layer}: {len(hashed_trajectory_layer)}"
        )
        for hashed_coordinate in hashed_trajectory_layer:
            folium.CircleMarker(hashed_coordinate, radius=10, color=colors[-(layer)-1]).add_to(map)
        layer += 1
    number_of_hashed_trajectories += 1

folium.TileLayer(tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
                 attr='Map data © <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', 
                 name='Transparent Tiles', 
                 opacity=0.2).add_to(map)  # You can adjust opacity here
map

Trajectory hash: R_ATG -> [[[41.894523201945674, 12.444897143771144], [41.894523201945674, 12.452136289574641], [41.89994943234342, 12.452136289574641], [41.89994943234342, 12.459375435378137], [41.89994943234342, 12.466614581181634], [41.89994943234342, 12.473853726985132], [41.894523201945674, 12.473853726985132], [41.89994943234342, 12.473853726985132]], [[41.89771181087993, 12.441911928098412], [41.89771181087993, 12.44915107390191], [41.89771181087993, 12.456390219705405], [41.89771181087993, 12.463629365508902], [41.89771181087993, 12.4708685113124], [41.90313804127768, 12.4708685113124], [41.89771181087993, 12.4708685113124], [41.90313804127768, 12.4708685113124], [41.89771181087993, 12.4708685113124], [41.90313804127768, 12.4708685113124], [41.89771181087993, 12.4708685113124]]]
Length of hashed coordinate in layer 0: 8
Length of hashed coordinate in layer 1: 11
Trajectory hash: R_AHN -> [[[41.905375662741164, 12.495571164395622], [41.89994943234342, 12.495571164395622], [41.90